In [ ]:
from kfold_and_metrics import *

import tensorflow as tf
import pandas as pd
import numpy as np

tf.keras.backend.clear_session()

In [ ]:
df = pd.read_csv("final.csv")
df = df.drop(columns=['id'])
df.head()

In [ ]:
activation_possibilities = ["softmax", "relu", "sigmoid"]
best_auc = 0
best = {}
for activation_hidden in activation_possibilities:
    for num_nodes_hidden_layer in range(5, len(df.columns)*2//3, 10):
        for gradient in [0.001, 0.003, 0.005, 0.007, 0.01]:
            params = {'hidden_layer_nodes': num_nodes_hidden_layer, 'hidden_layer_activation': activation_hidden, 'learning_rate': gradient}
            print("Current parameter combination:")
            for parameter, value in params.items():
                print(f"\t{parameter}: {value}")
            print()

            nn_model = tf.keras.models.Sequential([
                tf.keras.layers.Input((len(df.columns)-2,), name="input"),
                tf.keras.layers.Dense(num_nodes_hidden_layer,activation=activation_hidden),
                tf.keras.layers.Dense(2,activation='softmax')
            ])
            nn_model.compile(
                optimizer=tf.keras.optimizers.SGD(gradient), 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
            )
            score = k_fold_cv_keras(compiled_model=nn_model, df=df)
            results = mean_std_results_k_fold_CV(score)

            auc_avg = results.iloc[2,1]
            if auc_avg > best_auc:
                best_auc = auc_avg
                best = params

print("Results of the grid search parameter hypertunning:")
for parameter, value in best.items():
    print(f"\t{parameter}: {value}")